# Time series representation and analysis

The [astropy.timeseries](http://docs.astropy.org/en/stable/timeseries/index.html) sub-package is a relatively new component of astropy which provides a framework for representing time series, and includes some initial analysis tools (but this is expected to grow over time).


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create time series</li>
<li>Access data and times</li>
<li>Relative time series</li>
<li>Combining time series</li>
<li>Sorting time series</li>
<li>Folding time series</li>
<li>Periodogram analysis</li>
<li>Reading in data</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.timeseries. For more information about the features presented below as well as other available features, you can read the
[astropy.timeseries documentation](https://docs.astropy.org/en/stable/timeseries/).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Creating time series

The astropy.timeseries sub-package provides two main classes for representing time series, both of which are sub-classes of [QTable](http://docs.astropy.org/en/stable/api/astropy.table.QTable.html#astropy.table.QTable):

* [TimeSeries](http://docs.astropy.org/en/stable/api/astropy.timeseries.TimeSeries.html#astropy.timeseries.TimeSeries) to represent a series of measurements at different (and exact) times
* [BinnedTimeSeries](http://docs.astropy.org/en/stable/api/astropy.timeseries.BinnedTimeSeries.html#astropy.timeseries.BinnedTimeSeries), to represent a series of measurements in different time bins.

In most cases, [TimeSeries](http://docs.astropy.org/en/stable/api/astropy.timeseries.TimeSeries.html#astropy.timeseries.TimeSeries) might be enough unless you need to treat the finite bin sizes carefully (as for example in X-ray analysis). In this tutorial we will focus on TimeSeries, but if you are interested in representing binned time series, you can take a look at the [astropy.timeseries documentation](http://docs.astropy.org/en/stable/timeseries/index.html).

Let's start off by creating a synthetic time series, and we will look later into how to read existing files.

In [2]:
from astropy.timeseries import TimeSeries

There are several ways to initialize a time series from scratch. One of the easiest ways if you are dealing with a regularly spaced time series is to specify the start time, the time between samples, and the number of samples:

In [3]:
from astropy import units as u
ts1 = TimeSeries(time_start='2016-03-22T12:30:31', time_delta=3*u.s, n_samples=5)
ts1

<TimeSeries length=5>
          time         
         object        
-----------------------
2016-03-22T12:30:31.000
2016-03-22T12:30:34.000
2016-03-22T12:30:37.000
2016-03-22T12:30:40.000
2016-03-22T12:30:43.000

Another way is to specify the times at which the measurements were made:

In [4]:
ts2 = TimeSeries(time=['2016-03-22T12:30:30', '2016-03-22T12:30:38', '2016-03-22T12:30:40'])
ts2

<TimeSeries length=3>
          time         
         object        
-----------------------
2016-03-22T12:30:30.000
2016-03-22T12:30:38.000
2016-03-22T12:30:40.000

In the above cases, we just passed strings to the ``time=`` option, but this is actually a shortcut - the most general way to create the time series which provides the most control is to pass a Time object (from astropy.time):

In [6]:
import numpy as np
from astropy.time import Time
time = Time(np.linspace(50000, 51000, 6), format='mjd', scale='tdb')
ts3 = TimeSeries(time=time)
ts3

<TimeSeries length=6>
  time 
 object
-------
50000.0
50200.0
50400.0
50600.0
50800.0
51000.0

Once the time series has been initialized, you can add columns to it as if it was a table:

In [7]:
ts3['flux'] = [1,4,3,2,1,3] * u.mJy
ts3['fluxerr'] = [0.1, 0.2, 0.2, 0.1, 0.1, 0.2] * u.mJy
ts3

<TimeSeries length=6>
  time    flux  fluxerr
          mJy     mJy  
 object float64 float64
------- ------- -------
50000.0     1.0     0.1
50200.0     4.0     0.2
50400.0     3.0     0.2
50600.0     2.0     0.1
50800.0     1.0     0.1
51000.0     3.0     0.2

## Accessing data and times

Data is accessed in the same way as for regular tables:

In [8]:
ts3['time']

<Time object: scale='tdb' format='mjd' value=[50000. 50200. 50400. 50600. 50800. 51000.]>

In [9]:
ts3['flux']

<Quantity [1., 4., 3., 2., 1., 3.] mJy>

In [10]:
ts3['time', 'flux']

<TimeSeries length=6>
  time    flux 
          mJy  
 object float64
------- -------
50000.0     1.0
50200.0     4.0
50400.0     3.0
50600.0     2.0
50800.0     1.0
51000.0     3.0

In [15]:
ts3[0]

<Row index=0>
  time    flux  fluxerr
          mJy     mJy  
 object float64 float64
------- ------- -------
50000.0     1.0     0.1

In [17]:
ts3[0:3]

<TimeSeries length=3>
  time    flux  fluxerr
          mJy     mJy  
 object float64 float64
------- ------- -------
50000.0     1.0     0.1
50200.0     4.0     0.2
50400.0     3.0     0.2

In [11]:
ts3[0]['flux']

<Quantity 1. mJy>

For the time column, it is also possible to use the following shortcut:

In [12]:
ts3.time

<Time object: scale='tdb' format='mjd' value=[50000. 50200. 50400. 50600. 50800. 51000.]>

Since this is a regular time object, you can convert it to different time scales and formats:

In [13]:
ts3.time.tai

<Time object: scale='tai' format='mjd' value=[49999.99962752 50199.99962748 50399.99962752 50599.99962749
 50799.99962751 50999.9996275 ]>

In [14]:
ts3.time.isot

array(['1995-10-10T00:00:00.000', '1996-04-27T00:00:00.000',
       '1996-11-13T00:00:00.000', '1997-06-01T00:00:00.000',
       '1997-12-18T00:00:00.000', '1998-07-06T00:00:00.000'], dtype='<U23')

Note that this doesn't change the time in the time series, it just returns a new Time object. On the other hand, you can change the default format by setting ``.format``:

In [18]:
ts3

<TimeSeries length=6>
  time    flux  fluxerr
          mJy     mJy  
 object float64 float64
------- ------- -------
50000.0     1.0     0.1
50200.0     4.0     0.2
50400.0     3.0     0.2
50600.0     2.0     0.1
50800.0     1.0     0.1
51000.0     3.0     0.2

In [19]:
ts3.time.format = 'isot'
ts3

<TimeSeries length=6>
          time            flux  fluxerr
                          mJy     mJy  
         object         float64 float64
----------------------- ------- -------
1995-10-10T00:00:00.000     1.0     0.1
1996-04-27T00:00:00.000     4.0     0.2
1996-11-13T00:00:00.000     3.0     0.2
1997-06-01T00:00:00.000     2.0     0.1
1997-12-18T00:00:00.000     1.0     0.1
1998-07-06T00:00:00.000     3.0     0.2

## Relative time series

If you want to work with relative time series, you can do the following:

In [20]:
from astropy.time import TimeDelta
ts4 = TimeSeries(time=TimeDelta([1, 3, 4] * u.s))
ts4

<TimeSeries length=3>
         time         
        object        
----------------------
1.1574074074074073e-05
 3.472222222222222e-05
4.6296296296296294e-05

and this can also be done by taking a time series in absolute times and subtracting a reference time:

In [21]:
ts5 = TimeSeries(time=ts3.time - ts3.time[0], data = ts3['flux', 'fluxerr'])
ts5

<TimeSeries length=6>
 time    flux  fluxerr
         mJy     mJy  
object float64 float64
------ ------- -------
   0.0     1.0     0.1
 200.0     4.0     0.2
 400.0     3.0     0.2
 600.0     2.0     0.1
 800.0     1.0     0.1
1000.0     3.0     0.2

## Combining time series

Assuming that you have two time series objects with matching columns as well as another table which represents an additional column:

In [22]:
from astropy.table import Table

ts_a = TimeSeries(time_start='2016-03-22T12:30:31',
                  time_delta=3 * u.s,
                  data={'flux': [1, 4, 5, 3, 2] * u.mJy})

ts_b = TimeSeries(time_start='2016-03-22T12:50:31',
                  time_delta=3 * u.s,
                  data={'flux': [4, 3, 1, 2, 3] * u.mJy})

data = Table(data={'temperature': [40., 41., 40., 39., 30.] * u.K})

In [27]:
ts_a

<TimeSeries length=5>
          time            flux 
                          mJy  
         object         float64
----------------------- -------
2016-03-22T12:30:31.000     1.0
2016-03-22T12:30:34.000     4.0
2016-03-22T12:30:37.000     5.0
2016-03-22T12:30:40.000     3.0
2016-03-22T12:30:43.000     2.0

In [28]:
ts_b

<TimeSeries length=5>
          time            flux 
                          mJy  
         object         float64
----------------------- -------
2016-03-22T12:50:31.000     4.0
2016-03-22T12:50:34.000     3.0
2016-03-22T12:50:37.000     1.0
2016-03-22T12:50:40.000     2.0
2016-03-22T12:50:43.000     3.0

In [29]:
data

<Table length=5>
temperature
     K     
  float64  
-----------
       40.0
       41.0
       40.0
       39.0
       30.0

There are different ways of combining these - for example you can stack one of the time series with the additional data 'horizontally', which means you preserve the number of rows but construct a new time series with more columns:

In [30]:
from astropy.table import hstack, vstack

In [32]:
tsh = hstack([ts_a, data])
tsh

<TimeSeries length=5>
          time            flux  temperature
                          mJy        K     
         object         float64   float64  
----------------------- ------- -----------
2016-03-22T12:30:31.000     1.0        40.0
2016-03-22T12:30:34.000     4.0        41.0
2016-03-22T12:30:37.000     5.0        40.0
2016-03-22T12:30:40.000     3.0        39.0
2016-03-22T12:30:43.000     2.0        30.0

or you can stack the time series 'vertically' meaning that the columns are preserved but we make a new time series with all the rows from the original two time series:

In [38]:
tsv = vstack([ts_b, ts_a])
tsv

<TimeSeries length=10>
          time            flux 
                          mJy  
         object         float64
----------------------- -------
2016-03-22T12:50:31.000     4.0
2016-03-22T12:50:34.000     3.0
2016-03-22T12:50:37.000     1.0
2016-03-22T12:50:40.000     2.0
2016-03-22T12:50:43.000     3.0
2016-03-22T12:30:31.000     1.0
2016-03-22T12:30:34.000     4.0
2016-03-22T12:30:37.000     5.0
2016-03-22T12:30:40.000     3.0
2016-03-22T12:30:43.000     2.0

In [41]:
tsv.sort('time')
tsv

<TimeSeries length=10>
          time            flux 
                          mJy  
         object         float64
----------------------- -------
2016-03-22T12:30:31.000     1.0
2016-03-22T12:30:34.000     4.0
2016-03-22T12:30:37.000     5.0
2016-03-22T12:30:40.000     3.0
2016-03-22T12:30:43.000     2.0
2016-03-22T12:50:31.000     4.0
2016-03-22T12:50:34.000     3.0
2016-03-22T12:50:37.000     1.0
2016-03-22T12:50:40.000     2.0
2016-03-22T12:50:43.000     3.0

## Sorting

Since TimeSeries objects are just tables behind the hood, you can also sort them as you would a table:

In [36]:
tsv.sort('flux')
tsv

<TimeSeries length=10>
          time            flux 
                          mJy  
         object         float64
----------------------- -------
2016-03-22T12:30:31.000     1.0
2016-03-22T12:50:37.000     1.0
2016-03-22T12:30:43.000     2.0
2016-03-22T12:50:40.000     2.0
2016-03-22T12:30:40.000     3.0
2016-03-22T12:50:34.000     3.0
2016-03-22T12:50:43.000     3.0
2016-03-22T12:30:34.000     4.0
2016-03-22T12:50:31.000     4.0
2016-03-22T12:30:37.000     5.0

## Folding

A common operation with time series is 'folding' by assuming a given period. Let's start off with a synthetic dataset:

In [42]:
ts_synth = TimeSeries(time_start='2019-01-01T00:00:00',
                      time_delta=np.random.uniform(0, 10, 1000) * u.min)
ts_synth['flux'] = np.cos(2 * np.pi * (ts_synth.time.mjd / 0.49822))
ts_synth['flux'] += np.random.normal(0, 0.2, 1000)

In [44]:
plt.plot(ts_synth.time.mjd, ts_synth['flux'], 'o');

Folding is done with the [TimeSeries.fold](http://docs.astropy.org/en/stable/api/astropy.timeseries.TimeSeries.html#astropy.timeseries.TimeSeries.fold) method:

In [45]:
folded = ts_synth.fold(period=0.49822 * u.day)
folded

<TimeSeries length=1000>
         time                 flux       
        object              float64      
--------------------- -------------------
                  0.0  0.6051802949907192
 0.005248601950405937   0.829149333708596
 0.010955730054515276  0.7307510851129585
  0.01654649932908468   0.574636800383557
  0.01761229533585484   1.093116277322885
 0.023444153923241833  0.7407016130994658
 0.027767414550486366  1.3156945314190804
 0.031493534221127883  1.0082479983026937
 0.032264304556881764  0.5488446377286036
 0.034673749769606146   0.770273355827187
                  ...                 ...
 -0.05406336289750058 0.08203196206119648
 -0.04876963240310395  0.3455610653490189
 -0.04835776204339633  0.3004142106843401
-0.044697376776433866  0.2827644235002205
 -0.04381336014931439 0.07464808559919256
 -0.04350859099431402  0.2578089920019343
 -0.04163084393620205  0.6504684234751714
 -0.04112532465297885 0.10117368649593633
 -0.03671435032707307  0.6644493670973721
 -0.03500384083434786 0.16578308350345994

In [47]:
folded.time.value[:10]

array([0.        , 0.0052486 , 0.01095573, 0.0165465 , 0.0176123 ,
       0.02344415, 0.02776741, 0.03149353, 0.0322643 , 0.03467375])

In [49]:
plt.plot(folded.time.value, ts_synth['flux'], '.');

## Periodogram analysis

In the above example it was clear what period to use since we generated the data, but how would we find the period in the first place if it wasn't known? The astropy.timeseries sub-package includes several algorithms for periodogram analysis, the most general one being the Lomb-Scargle algorithm ([Lomb 1976](http://adsabs.harvard.edu/abs/1976Ap%26SS..39..447L), [Scargle 1982](http://adsabs.harvard.edu/abs/1982ApJ...263..835S)). To use this, first create an instance of the [LombScargle](http://docs.astropy.org/en/stable/api/astropy.timeseries.LombScargle.html#astropy.timeseries.LombScargle) class:

In [50]:
from astropy.timeseries import LombScargle

In [51]:
ls = LombScargle.from_timeseries(ts_synth, 'flux')

To run the periodogram analysis, the easiest way is to call the [autopower](http://docs.astropy.org/en/stable/api/astropy.timeseries.LombScargle.html#astropy.timeseries.LombScargle.autopower) method:

In [52]:
frequency, power = ls.autopower()

In [53]:
frequency

<Quantity [2.89642151e-02, 8.68926453e-02, 1.44821076e-01, ...,
           7.24018485e+02, 7.24076414e+02, 7.24134342e+02] 1 / d>

In [54]:
power

<Quantity [0.00649531, 0.00583384, 0.00446988, ..., 0.00074942, 0.0017767 ,
           0.0031457 ]>

In [55]:
plt.plot(frequency, power);

We can zoom in on the low frequency peak by doing:

In [59]:
frequency, power = ls.autopower(minimum_frequency=0.01/u.day, maximum_frequency=10/u.day)
plt.plot(frequency, power);

The peak is located at:

In [57]:
peak = np.argmax(power)
frequency[peak]

<Quantity 1.99853084 1 / d>

In [60]:
1/ frequency[peak]

<Quantity 0.50516107 d>

## Reading in data

Reading in time series can be done via the [TimeSeries.read](http://docs.astropy.org/en/stable/api/astropy.timeseries.TimeSeries.html#astropy.timeseries.TimeSeries.read) method. This accepts all the formats supported by the Table class, and requires that you specify the name of the time column, and optionally the format and scale for the time. We will take a look at this by reading a CSV file that looks like:

    Date,A,B,C,D,E,F,G
    2008-03-18,24.68,164.93,114.73,26.27,19.21,28.87,63.44
    2008-03-19,24.18,164.89,114.75,26.22,19.07,27.76,59.98
    ...
    
To read this file, you can do:

In [63]:
TimeSeries.read('data/sampled.csv', time_column='Date')

<TimeSeries length=11>
          time             A       B       C       D       E       F       G   
         object         float64 float64 float64 float64 float64 float64 float64
----------------------- ------- ------- ------- ------- ------- ------- -------
2008-03-18 00:00:00.000   24.68  164.93  114.73   26.27   19.21   28.87   63.44
2008-03-19 00:00:00.000   24.18  164.89  114.75   26.22   19.07   27.76   59.98
2008-03-20 00:00:00.000   23.99  164.63  115.04   25.78   19.01   27.04   59.61
2008-03-25 00:00:00.000   24.14  163.92  114.85   27.41   19.61   27.84   59.41
2008-03-26 00:00:00.000   24.44  163.45  114.84   26.86   19.53   28.02   60.09
2008-03-27 00:00:00.000   24.38  163.46   115.4   27.09   19.72   28.25   59.62
2008-03-28 00:00:00.000   24.32  163.22  115.56   27.13   19.63   28.24   58.65
2008-03-31 00:00:00.000   24.19  164.02  115.54   26.74   19.55   28.43    59.2
2008-04-01 00:00:00.000   23.81  163.59  115.72   27.82   20.21   29.17   56.18
2008-04-02 00:00:00.000   24.03  163.32  115.11   28.22   20.42   29.38   56.64
2008-04-03 00:00:00.000   24.34  163.34  115.17   28.14   20.36   29.51   57.49

For some well defined formats (e.g. Kepler/TESS time series files), it is enough to specify the filename.

<center><i>This notebook was originally written by <a href="https://aperiosoftware.com/">Aperio Software Ltd</a> and modified by Brigitta Sip≈ëcz. &copy; 2019-2021 and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)